### <b> Chuẩn bị

### Importing libararies

In [138]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

### Import dataset


In [139]:
data = pd.read_csv('dataset.csv', header = 0)
df = pd.DataFrame(data)
df.head()

,local_ref_1,vn_marital_status,resid_province,resid_district,resid_wards,birth_incorp_date,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,...,totalSavings2021_heoSo,balanceJuin2021,nominal_interestJuin2021,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021
0,MALE,Married,27.0,262.0,9436.0,1992.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MALE,Single,79.0,769.0,27097.0,1986.0,NaN,NaN,NaN,NaN,...,NaN,0.003666,0.006953,0.004128,1.0,21061.0,365.0,DH,1890.0,22321.0
2,MALE,Married,1.0,250.0,8989.0,1980.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MALE,Single,40.0,431.0,18055.0,2001.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MALE,Married,49.0,512.0,20758.0,1989.0,0.0,0.000114,0.000636,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Xử lý dữ liệu

### Kích thước dữ liệu


In [140]:
df.shape

(100000, 50)

### Kiểu dữ liệu

In [141]:
df.dtypes

local_ref_1                      object
vn_marital_status                object
resid_province                  float64
resid_district                  float64
resid_wards                     float64
birth_incorp_date               float64
 amount_week                    float64
max_amount_week                 float64
min_amount_week                 float64
distinct_payment_code_week      float64
count_payment_code_week         float64
distinct_trans_group_week       float64
distinct_ref_no_week            float64
amount_month                    float64
max_amount_month                float64
min_amount_month                float64
distinct_payment_code_month     float64
count_payment_code_month        float64
distinct_trans_group_month      float64
distinct_ref_no_month           float64
amount_3month                   float64
max_amount_3month               float64
min_amount_3month               float64
distinct_payment_code_3month      int64
count_payment_code_3month         int64


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   local_ref_1                   97266 non-null   object 
 1   vn_marital_status             96971 non-null   object 
 2   resid_province                93431 non-null   float64
 3   resid_district                93435 non-null   float64
 4   resid_wards                   93433 non-null   float64
 5   birth_incorp_date             99990 non-null   float64
 6    amount_week                  54800 non-null   float64
 7   max_amount_week               54800 non-null   float64
 8   min_amount_week               54800 non-null   float64
 9   distinct_payment_code_week    56850 non-null   float64
 10  count_payment_code_week       56850 non-null   float64
 11  distinct_trans_group_week     56850 non-null   float64
 12  distinct_ref_no_week          56850 non-null 

- Tên trường "amount_week" và "total_amt_mar2021" bị thừa khoảng trắng

In [143]:
df = df.rename(columns={' amount_week ': 'amount_week'})
df = df.rename(columns={' total_amt_mar2021 ': 'total_amt_mar2021'})

- Check missing values

In [144]:
df.isnull().sum()

local_ref_1                      2734
vn_marital_status                3029
resid_province                   6569
resid_district                   6565
resid_wards                      6567
birth_incorp_date                  10
amount_week                     45200
max_amount_week                 45200
min_amount_week                 45200
distinct_payment_code_week      43150
count_payment_code_week         43150
distinct_trans_group_week       43150
distinct_ref_no_week            43150
amount_month                    23000
max_amount_month                23000
min_amount_month                23000
distinct_payment_code_month     17777
count_payment_code_month        17777
distinct_trans_group_month      17777
distinct_ref_no_month           17777
amount_3month                   13480
max_amount_3month               13480
min_amount_3month               13480
distinct_payment_code_3month        0
count_payment_code_3month           0
distinct_trans_group_3month         0
distinct_ref

- Mã hóa nhị phân cột giới tính


In [145]:
df['Male'] = np.where(df['local_ref_1'] == 'MALE', 1, 0)
df.drop(['local_ref_1'], axis=1, inplace=True)

In [146]:
df['vn_marital_status'].value_counts()

vn_marital_status
Single       54399
Married      42126
Other          363
Partner         38
Divorced        33
Widow            8
Seperated        4
Name: count, dtype: int64

In [147]:
df['Single'] = np.where(df['vn_marital_status'] == 'Single', 1, 0)
df.drop(['vn_marital_status'], axis=1, inplace=True)

- Tính cột tuổi, xóa cột năm sinh, fill NaN bằng median

In [148]:
df['Age'] = 2021 - df['birth_incorp_date']
df.drop(['birth_incorp_date'], axis=1, inplace=True)

In [149]:
df['Age'].fillna(df['Age'].median(), inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_5348\1933487976.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)


- Kiểm tra giá trị cột "Age" không hợp lý

In [150]:
cond_age = df['Age'] > 100
df.loc[cond_age]

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
18897,75.0,740.0,26389.0,0.0,0.000081,0.000023,2.0,3.0,2.0,4.0,...,0.035203,1.0,21061.0,363.0,DH,1890.0,22312.0,1,0,242.0
39010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,116.0
47634,92.0,916.0,31117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,121.0
54895,15.0,135.0,4348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,137.0
73845,56.0,568.0,22363.0,0.0,0.000229,0.001272,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,948.0


- Xóa các hàng có giá trị cột "Age" không hợp lý

In [151]:
df = df[df['Age'] <= 100]

- Đối với năm 2020, các trường thông tin giao dịch của tuần trước có NaN được hiểu là không có giao dịch và fill NaN bằng 0 do missing values của các cột amount, min, max tương ứng với nhau, đối với các trường thông tin có giá trị 0 từ trước, fill NaN bằng mode
- Tương tự đối với giao dịch của 1 tháng trước và 3 tháng trước

In [152]:
df['distinct_payment_code_week'].value_counts()

distinct_payment_code_week
0.0    39093
1.0    16482
2.0     1163
3.0      101
4.0        8
5.0        1
Name: count, dtype: int64

In [153]:
df['count_payment_code_week'].value_counts()

count_payment_code_week
0.0      39093
1.0      10972
2.0       4044
3.0       1494
4.0        596
5.0        277
6.0        132
7.0         51
8.0         45
9.0         32
10.0        26
11.0        18
16.0         7
14.0         6
182.0        6
12.0         5
13.0         4
28.0         3
23.0         3
17.0         3
29.0         2
20.0         2
82.0         2
27.0         2
132.0        2
43.0         2
15.0         2
24.0         2
31.0         1
42.0         1
41.0         1
39.0         1
35.0         1
19.0         1
26.0         1
40.0         1
25.0         1
21.0         1
67.0         1
114.0        1
61.0         1
37.0         1
331.0        1
Name: count, dtype: int64

In [154]:
df['count_payment_code_week'].value_counts()

count_payment_code_week
0.0      39093
1.0      10972
2.0       4044
3.0       1494
4.0        596
5.0        277
6.0        132
7.0         51
8.0         45
9.0         32
10.0        26
11.0        18
16.0         7
14.0         6
182.0        6
12.0         5
13.0         4
28.0         3
23.0         3
17.0         3
29.0         2
20.0         2
82.0         2
27.0         2
132.0        2
43.0         2
15.0         2
24.0         2
31.0         1
42.0         1
41.0         1
39.0         1
35.0         1
19.0         1
26.0         1
40.0         1
25.0         1
21.0         1
67.0         1
114.0        1
61.0         1
37.0         1
331.0        1
Name: count, dtype: int64

In [155]:
df['distinct_trans_group_week'].value_counts()

distinct_trans_group_week
1.0    34899
2.0    17101
3.0     3932
4.0      788
5.0      118
6.0       10
Name: count, dtype: int64

In [156]:
df['distinct_ref_no_week'].value_counts()

distinct_ref_no_week
1.0       14202
2.0        9491
3.0        6677
4.0        4678
5.0        3426
          ...  
152.0         1
1128.0        1
148.0         1
149.0         1
84.0          1
Name: count, Length: 143, dtype: int64

In [157]:
def fill_na_for_2020(df, period):
    df[f'amount_{period}'].fillna(0, inplace=True)
    df[f'min_amount_{period}'].fillna(0, inplace=True)
    df[f'max_amount_{period}'].fillna(0, inplace=True)
    df[f'distinct_trans_group_{period}'].fillna(0, inplace=True)
    df[f'distinct_ref_no_{period}'].fillna(0, inplace=True)
    df[f'distinct_payment_code_{period}'].fillna(df[f'distinct_payment_code_{period}'].mode()[0], inplace=True)
    df[f'count_payment_code_{period}'].fillna(df[f'count_payment_code_{period}'].mode()[0], inplace=True)
    return df
periods = ['week', 'month', '3month']
for period in periods:
    fill_na_for_2020(df, period)
print("\nUpdated DataFrame:")
print(df)    

C:\Users\User\AppData\Local\Temp\ipykernel_5348\3673713568.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[f'amount_{period}'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_5348\3673713568.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 


Updated DataFrame:
       resid_province  resid_district  resid_wards  amount_week  \
0                27.0           262.0       9436.0          0.0   
1                79.0           769.0      27097.0          0.0   
2                 1.0           250.0       8989.0          0.0   
3                40.0           431.0      18055.0          0.0   
4                49.0           512.0      20758.0          0.0   
...               ...             ...          ...          ...   
99995             1.0             6.0        178.0          0.0   
99996            25.0           239.0       8689.0          0.0   
99997            31.0           316.0      11842.0          0.0   
99998             1.0           271.0       9706.0          0.0   
99999            26.0           251.0       9070.0          0.0   

       max_amount_week  min_amount_week  distinct_payment_code_week  \
0             0.000000         0.000000                         0.0   
1             0.000000         0.

In [158]:
df.head()

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
0,27.0,262.0,9436.0,0.0,0.000000,0.000000,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,29.0
1,79.0,769.0,27097.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.004128,1.0,21061.0,365.0,DH,1890.0,22321.0,1,1,35.0
2,1.0,250.0,8989.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,41.0
3,40.0,431.0,18055.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,20.0
4,49.0,512.0,20758.0,0.0,0.000114,0.000636,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,32.0


- Một số hàng có giá trị cột số người nhận giao dịch lớn hơn 0 nhưng tổng số tiền giao dịch bằng 0

In [159]:
cond_ref_week = (df['distinct_ref_no_week'] > 0) & (df['max_amount_week'] == 0)
df.loc[cond_ref_week]

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
0,27.0,262.0,9436.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,29.0
91,38.0,406.0,16510.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,28.0
103,38.0,395.0,15484.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,29.0
157,10.0,88.0,3001.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,22.0
327,1.0,6.0,178.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99986,1.0,275.0,9925.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,24.0
99988,24.0,222.0,7807.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,21.0
99989,1.0,274.0,9856.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,17.0
99996,25.0,239.0,8689.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,20.0


In [160]:
cond_ref_month = (df['distinct_ref_no_month'] > 0) & (df['max_amount_month'] == 0)
df.loc[cond_ref_month]

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
46,1.0,6.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,29.0
57,87.0,874.0,30157.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,19.0
64,19.0,164.0,5464.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,19.0
86,1.0,6.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,21.0
103,38.0,395.0,15484.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99986,1.0,275.0,9925.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,24.0
99988,24.0,222.0,7807.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,21.0
99989,1.0,274.0,9856.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,17.0
99996,25.0,239.0,8689.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,20.0


In [161]:
cond_ref_3month = (df['distinct_ref_no_3month'] > 0) & (df['max_amount_3month'] == 0)
df.loc[cond_ref_3month]

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
3,40.0,431.0,18055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,20.0
6,1.0,6.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,27.0
9,1.0,6.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,21.0
13,33.0,323.0,11965.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,37.0
15,37.0,369.0,14320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99990,54.0,555.0,22041.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,18.0
99995,1.0,6.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,29.0
99996,25.0,239.0,8689.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,20.0
99998,1.0,271.0,9706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,19.0


- Ở một số hàng cột "min_amount_week" có giá trị lớn hơn cột "max_amount_week", có thể do đơn vị của 2 cột khác nhau, cần lưu ý khi đọc kết quả mô hình

In [162]:
check = (df['min_amount_week'] > df['max_amount_week']).sum()
check

23680

In [163]:
check = (df['min_amount_month'] > df['max_amount_month']).sum()
check

9497

In [164]:
check = (df['min_amount_3month'] > df['max_amount_3month']).sum()
check

9803

- Một số hàng giá trị cột tổng số tiền giao dịch nhỏ hơn tổng số tiền giao dịch lớn nhất và nhỏ nhất, có thể do đơn vị của các cột khác nhau, cần lưu ý khi đọc kết quả mô hình

In [165]:
cond_amount_week = df['amount_week'] < df['min_amount_week'] + df['max_amount_week']
df.loc[cond_amount_week]

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
4,49.0,512.0,20758.0,0.0,0.000114,0.000636,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,32.0
7,38.0,394.0,15436.0,0.0,0.000476,0.000011,0.0,0.0,1.0,11.0,...,0.000000,1.0,1005.0,379.0,NH,1740.0,23311.0,1,0,29.0
12,56.0,568.0,22357.0,0.0,0.000014,0.000079,0.0,0.0,1.0,1.0,...,0.000000,1.0,17001.0,NaN,NH,1740.0,26111.0,1,1,25.0
17,92.0,925.0,31264.0,0.0,0.000449,0.001135,0.0,0.0,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,31.0
18,38.0,399.0,15874.0,0.0,0.001429,0.000681,0.0,0.0,1.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99956,83.0,829.0,28780.0,0.0,0.000031,0.000172,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,31.0
99976,33.0,330.0,12268.0,0.0,0.000408,0.002271,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,30.0
99980,70.0,689.0,25210.0,0.0,0.000036,0.000203,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,18.0
99985,1.0,250.0,8977.0,0.0,0.000008,0.000045,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,36.0


In [166]:
cond_amount_month = df['amount_month'] < df['min_amount_month'] + df['max_amount_month']
df.loc[cond_amount_month]

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
0,27.0,262.0,9436.0,0.0,0.000000,0.000000,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,29.0
1,79.0,769.0,27097.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.004128,1.0,21061.0,365.0,DH,1890.0,22321.0,1,1,35.0
2,1.0,250.0,8989.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,41.0
4,49.0,512.0,20758.0,0.0,0.000114,0.000636,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,32.0
5,37.0,377.0,14719.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99991,66.0,643.0,24124.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,34.0
99992,36.0,363.0,14047.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,22.0
99993,79.0,778.0,27475.0,0.0,0.001633,0.009083,0.0,0.0,1.0,1.0,...,0.002533,1.0,21061.0,365.0,DH,1700.0,22321.0,0,0,32.0
99994,34.0,336.0,12469.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,29.0


In [167]:
cond_amount_3month = df['amount_3month'] < df['min_amount_3month'] + df['max_amount_3month']
df.loc[cond_amount_3month]

,resid_province,resid_district,resid_wards,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,distinct_ref_no_week,...,real_interestJuin2021,nhomno_xhtdJuin2021,categoryJuin2021,sub_productJuin2021,loaikyhanJuin2021,sectorJuin2021,product_codeJuin2021,Male,Single,Age
0,27.0,262.0,9436.0,0.0,0.000000,0.000000,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,29.0
1,79.0,769.0,27097.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.004128,1.0,21061.0,365.0,DH,1890.0,22321.0,1,1,35.0
2,1.0,250.0,8989.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,41.0
4,49.0,512.0,20758.0,0.0,0.000114,0.000636,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,32.0
5,37.0,377.0,14719.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99991,66.0,643.0,24124.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,34.0
99992,36.0,363.0,14047.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,22.0
99993,79.0,778.0,27475.0,0.0,0.001633,0.009083,0.0,0.0,1.0,1.0,...,0.002533,1.0,21061.0,365.0,DH,1700.0,22321.0,0,0,32.0
99994,34.0,336.0,12469.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,29.0
